Evaluate Flair models for person names, orgs and locations using the Presidio Evaluator framework

Data = `generated_test_November 12 2019`

In [ ]:
from presidio_evaluator.data_generator import read_synth_dataset
%reload_ext autoreload
%autoreload 2

Select data for evaluation:

In [ ]:
synth_samples = read_synth_dataset("../../data/synth_dataset.txt")
print(len(synth_samples))
DATASET = synth_samples

In [ ]:
from collections import Counter
entity_counter = Counter()
for sample in DATASET:
    for tag in sample.tags:
        entity_counter[tag]+=1

In [ ]:
entity_counter

In [ ]:
DATASET[1]

In [ ]:
#max length sentence
max([len(sample.tokens) for sample in DATASET])

Select models for evaluation:

In [ ]:
flair_ner = 'ner'
flair_ner_fast = 'ner-fast'
flair_ontonotes = 'ner-ontonotes-fast'
flair_bert_embeddings = '../../models/presidio-ner/flair-bert-embeddings.pt'
glove_flair_embeddings = '../../models/presidio-ner/flair-embeddings.pt'
models = [flair_bert_embeddings, glove_flair_embeddings, flair_ner,flair_ner_fast,flair_ontonotes]

In [ ]:
from presidio_evaluator.flair_evaluator import FlairEvaluator

for model in models:
    print("-----------------------------------")
    print("Evaluating model {}".format(model))
    flair_evaluator = FlairEvaluator(model_path=model)
    evaluation_results = flair_evaluator.evaluate_all(DATASET)
    scores = flair_evaluator.calculate_score(evaluation_results)
    
     
    print("Confusion matrix:")
    print(scores.results)

    print("Precision and recall")
    scores.print()
    errors = scores.model_errors


Custom evaluation

#### False positives

1. Most false positive tokens:

In [ ]:
errors = scores.model_errors

from presidio_evaluator import ModelEvaluator
ModelEvaluator.most_common_fp_tokens(errors)#[model_error for model_error in errors if model_error.error_type =='FP']


In [ ]:
fps_df = ModelEvaluator.get_fps_dataframe(errors,entity=['PERSON'])
fps_df[['full_text','token','prediction']]

2. False negative examples

In [ ]:
ModelEvaluator.most_common_fn_tokens(errors,n=50, entity=['PERSON'])

More FN analysis

In [ ]:
fns_df = ModelEvaluator.get_fns_dataframe(errors,entity=['PERSON'])

In [ ]:
fns_df[['full_text','token','annotation','prediction']]